# Part 1: Generate dataset

In [1]:
import cv2

def generate_dataset():
    # Load the pre-trained face detector from OpenCV
    face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

    # Function to detect and crop the face from an image
    def face_cropped(img):
        # Convert the image to grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # Detect faces in the image
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)

        # If no faces are detected, return None
        if len(faces) == 0:
            return None
        # Crop the first detected face
        for (x, y, w, h) in faces:
            cropped_face = img[y:y + h, x:x + w]
        return cropped_face

    # Initialize the webcam (0 for built-in webcam, 1 for external)
    cap = cv2.VideoCapture(0)
    id = 1  # Assign an ID for the user
    img_id = 0  # Image counter

    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()
        # Check if the frame was captured successfully
        if not ret:
            print("Failed to capture image. Exiting...")
            break

        # If a face is detected and cropped
        cropped_face = face_cropped(frame)
        if cropped_face is not None:
            img_id += 1
            # Resize the cropped face to 200x200 pixels
            face = cv2.resize(cropped_face, (200, 200))
            # Convert the face to grayscale
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            # Save the image to the dataset folder
            file_name_path = "data/user." + str(id) + "." + str(img_id) + ".jpg"
            cv2.imwrite(file_name_path, face)
            # Display the image with the image ID
            cv2.putText(face, str(img_id), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
            cv2.imshow("Cropped face", face)

        # Break the loop when 'Enter' (ASCII 13) is pressed or after 200 images
        if cv2.waitKey(1) == 13 or img_id == 200:
            break

    # Release the webcam and close all OpenCV windows
    cap.release()
    cv2.destroyAllWindows()
    print("Collecting samples is completed....")


# Call the function to start collecting face data
generate_dataset()


# Part 2: Train the classifier and save it.

In [2]:
import os
import cv2
from PIL import Image  # pip install pillow
import numpy as np  # pip install numpy


def train_classifier(data_dir):
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]

    faces = []
    ids = []

    for image in path:
        img = Image.open(image).convert('L')
        imageNp = np.array(img, 'uint8')
        id = int(os.path.split(image)[1].split(".")[1])

        faces.append(imageNp)
        ids.append(id)

    ids = np.array(ids)

    # Train and save classifier
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces, ids)
    clf.write("classifier.xml")


train_classifier("data")

# Part 3: Face detection/recognition

In [1]:
import cv2

def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, text, clf):
    # Convert the image to grayscale
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Detect features in the image
    features = classifier.detectMultiScale(gray_img, scaleFactor, minNeighbors)

    # Loop through detected features
    for (x, y, w, h) in features:
        # Draw rectangle around the detected face
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)

        # Predict the ID of the face
        id, pred = clf.predict(gray_img[y:y + h, x:x + w])
        confidence = int(100 * (1 - pred / 300))

        # Check confidence level and assign labels
        if confidence > 70:
            if id == 1:
                cv2.putText(img, "Vikas", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
            elif id == 2:
                cv2.putText(img, "Manish", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
        else:
            # If confidence is low, label as "UNKNOWN"
            cv2.putText(img, "UNKNOWN", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 1, cv2.LINE_AA)

    return img

# Load face classifier
faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

# Load the pre-trained face recognizer model
clf = cv2.face.LBPHFaceRecognizer_create()
clf.read("classifier.xml")

# Start video capture from the default camera (index 0)
video_capture = cv2.VideoCapture(0)

# Check if video capture is opened
if not video_capture.isOpened():
    print("Error: Could not open video stream.")
    exit()

# Loop to continuously get video frames
while True:
    # Read frame-by-frame
    ret, img = video_capture.read()

    # If frame capture fails, break the loop
    if not ret:
        print("Failed to capture image")
        break

    # Detect faces and draw boundaries
    img = draw_boundary(img, faceCascade, 1.3, 6, (255, 255, 255), "Face", clf)

    # Display the resulting frame
    cv2.imshow("Face Detection", img)

    # Break the loop if the 'Enter' key is pressed (ASCII value 13)
    if cv2.waitKey(1) == 13:
        break

# Release the video capture and close all windows
video_capture.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 